In [3]:
import os
from dotenv import load_dotenv
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor
from geoalchemy2 import Geometry
from sqlalchemy import create_engine
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import SQLDatabaseChain


In [4]:
load_dotenv()

True

In [5]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DATABASE_NAME = os.getenv("DATABASE_NAME")
DATABASE_USER = os.getenv("DATABASE_USER")
DATABASE_PASSWORD = os.getenv("DATABASE_PASSWORD")
DATABASE_PORT = os.getenv("DATABASE_PORT")
DATABASE_HOST = os.getenv("DATABASE_HOST")

In [6]:
llm=ChatOpenAI(model_name="gpt-4")

In [12]:
connection_string = f"postgresql://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}"
engine = create_engine(connection_string)

db = SQLDatabase(engine=engine)
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [7]:
connection_string = f"postgresql://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}"
engine = create_engine(connection_string)

db = SQLDatabase(engine=engine)
toolkit = SQLDatabaseToolkit(db=db)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [13]:
db_chain.run("how many tables are there?")



> Entering new SQLDatabaseChain chain...
how many tables are there? 
SQLQuery:

In [8]:
agent_executor.run("How many tables are there in the database?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: 
Observation: restriction, treestand, treestratum, assortment, operation, datasource, treestandsummary, specialfeature, stand, stand_4326, specification
Thought:I now know the final answer.
Final Answer: There are 11 tables in the database.

> Finished chain.


'There are 11 tables in the database.'

In [ ]:
agent_executor.run("What's the total sum of areas in the stand_4326 table?")